# Spatial correlation analysis with Pysal

## Readying and tidying data

In [1]:
# this cell contains our library imports
import pandas as pd
import altair as alt
import geopandas as geopandas
import contextily as cx
import folium as folium
from pysal.viz import mapclassify
from libpysal import weights
from splot.libpysal import plot_spatial_weights
import esda as esda

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
data = pd.read_csv('brussels-data-export.csv', sep = ';', decimal = ',')

In [3]:
data = data.query('fiscal_income > 0')

In [4]:
districts = geopandas.read_file('URBIS_ADM_MD/UrbAdm_MONITORING_DISTRICT.shp')

In [5]:
districts_joined = districts.merge(data, left_on = 'ID', right_on = 'code')

## Spatial weights & global autocorrelation

In [6]:
districts_weights_distance = weights.DistanceBand.from_dataframe(districts_joined, 2500)
districts_weights_distance.transform = 'R'
districts_joined['lagged_distance_fiscal_income'] = weights.lag_spatial(districts_weights_distance, districts_joined['fiscal_income'])

In [7]:
morans_i = esda.Moran(districts_joined['monthly_rent'], districts_weights_distance)
morans_i.I, morans_i.p_sim

(0.4562710057628412, 0.001)

In [8]:
districts_joined['lagged_distance_fiscal_income'] = weights.lag_spatial(districts_weights_distance, districts_joined['fiscal_income'])

In [9]:
chart = alt.Chart(districts_joined)
chart.mark_point().encode(
    x = 'fiscal_income',
    y = 'lagged_distance_fiscal_income'
)

alt.Chart(...)

## Local autocorrelation